In [1]:
import numpy as np
from math import ceil

from lockstar_rpi.BackendSettings import BackendSettings

In [9]:
    def calculate_prescaler_counter(sampling_rate):
        """Calculate prescaler and counter for a given sampling_rate. The MC realizes a sampling rate by scaling down 
        the internal clock_frequency (BackendSettings.mc_sampling_clock_rate) with the prescaler and then counting up to
        <counter> --> sampling_rate = internal_clock_rate / prescaler / counter"""
        rate = BackendSettings.mc_sampling_clock_rate / sampling_rate
        possible_prescalers = np.flip(np.arange(ceil(rate/BackendSettings.mc_max_counter), BackendSettings.mc_max_counter))
        possible_counters = rate/possible_prescalers
        best_counter = int(possible_counters[np.abs(possible_counters - possible_counters.astype(int)).argmin()])
        best_prescaler = int(rate/best_counter)
        print(possible_prescalers, possible_counters)
        return best_prescaler , best_counter

In [10]:
calculate_prescaler_counter(1000)

[65535 65534 65533 ...     7     6     5] [4.19623102e+00 4.19629505e+00 4.19635909e+00 ... 3.92857143e+04
 4.58333333e+04 5.50000000e+04]


(55000, 5)

In [12]:
bit="16"
TIM_BASE_CLOCK=275000000
TARGET_F = 1000
error=0.0
max_results=-1
DUTY=50.0

In [14]:
TARGET_ARR_MAX = { "8": 2**8-1, "16": 2**16-1, "32": 2**32-1 }
TARGET_PSC_MAX= { "16": 2**16-1 }
TARGET_PSC=0
TARGET_ARR=0



TARGET_UPDATE_F_MAX = TIM_BASE_CLOCK / ((TARGET_PSC + 1)*(TARGET_ARR+1))
TARGET_UPDATE_F_MIN = TIM_BASE_CLOCK / ((TARGET_PSC_MAX["16"] + 1)*(TARGET_ARR_MAX[bit]+1))
print("TARGET_UPDATE_F_MAX", TARGET_UPDATE_F_MAX)
print("TARGET_UPDATE_F_MIN", TARGET_UPDATE_F_MIN)

TARGET_UPDATE_F_MAX 275000000.0
TARGET_UPDATE_F_MIN 0.06402842700481415


In [15]:
results = []

def search_arr_psc():
    right = TIM_BASE_CLOCK / TARGET_F
    # right=int(right)
    print("(arr+1)*(psc+1) = TIM_BASE_CLOCK / TARGET_F")
    print("(arr+1)*(psc+1) =", TIM_BASE_CLOCK, "/", TARGET_F)
    print("(arr+1)*(psc+1) =",right)
    if TIM_BASE_CLOCK % TARGET_F != 0:
        print("\nSOLUTION WITHOUT ERROR NOT POSSIBLE - TIM_BASE_CLOCK / TARGET_F is not integer")
        if error == 0:
            print("You must use an --error= value >0 for this frequency!")
            quit()
    print( "Calculate...")
    for psc in range(0,TARGET_PSC_MAX["16"]+1):
        x = (TIM_BASE_CLOCK / (TARGET_F * (psc+1))) -1 
        if TIM_BASE_CLOCK  % (psc+1) == 0 and x <= TARGET_ARR_MAX[bit]:
            for arr in range(0,TARGET_ARR_MAX[bit]+1):
                left = (arr+1)*(psc+1) 
                pererror = abs((1- (TARGET_F / (TIM_BASE_CLOCK/left)))*100)
                if pererror <= error:
                    freq = TIM_BASE_CLOCK/left
                    d = int(arr/(100.0/DUTY))
                    results.append({"psc":psc,"arr":arr,"left":left,"freq":freq,"pererror":pererror,"delta":abs(freq-TARGET_F),"duty":d})
def secondlist(x):
    s = x
    ms = x*(10**3)
    us = x*(10**6)
    ns = x*(10**9)
    ps = x*(10**12)
    fs = x*(10**15)
    return {"s":s, "ms":ms, "µs":us, "ns":ns, "ps":ps, "fs":fs}


In [16]:
search_arr_psc()

if error != 0:
    result_sorted = sorted(results, key = lambda i: (i['pererror']))
else:
    result_sorted = sorted(results, key = lambda i: (i['psc']))

j=0
for i in result_sorted:
    if j == max_results:
        quit()
    print("PSC:",i["psc"],"ARR:",i["arr"]," => FREQ:",i["freq"],"[Hz]","ERROR:",i["pererror"],"[%] - CRRx[",DUTY,"%]:",i["duty"])
    j+=1
print (j,"Results. Reduce with --max=")

(arr+1)*(psc+1) = TIM_BASE_CLOCK / TARGET_F
(arr+1)*(psc+1) = 275000000 / 1000
(arr+1)*(psc+1) = 275000.0
Calculate...
PSC: 4 ARR: 54999  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 27499
PSC: 7 ARR: 34374  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 17187
PSC: 9 ARR: 27499  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 13749
PSC: 10 ARR: 24999  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 12499
PSC: 19 ARR: 13749  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 6874
PSC: 21 ARR: 12499  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 6249
PSC: 24 ARR: 10999  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 5499
PSC: 39 ARR: 6874  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 3437
PSC: 43 ARR: 6249  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 3124
PSC: 49 ARR: 5499  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 2749
PSC: 54 ARR: 4999  => FREQ: 1000.0 [Hz] ERROR: 0.0 [%] - CRRx[ 50.0 %]: 2499
PSC: 87 ARR: 3124  => FREQ